# Intro to LLMstudio


## start proxy server


In [1]:
from llmstudio.server import start_servers
start_servers()

Running LLMstudio Proxy on http://0.0.0.0:8001 
Running LLMstudio Tracking on http://0.0.0.0:8002 


In [2]:
from llmstudio_proxy.provider import LLMProxyProvider as LLM
from llmstudio_proxy.provider import ProxyConfig

llm = LLM(provider="openai", 
          proxy_config=ProxyConfig(host="0.0.0.0", port="8001"))

Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [3]:
result = llm.chat("olá", model="gpt-4o")


In [4]:
result.chat_output, result.metrics

('Olá! Como posso ajudá-lo hoje?',
 {'input_tokens': 2,
  'output_tokens': 11,
  'total_tokens': 13,
  'cost_usd': 0.000175,
  'latency_s': 0.9982657432556152,
  'time_to_first_token_s': 0.8587729930877686,
  'inter_token_latency_s': 0.01456777254740397,
  'tokens_per_second': 10.01737269615933})

### Import clients


In [5]:
from llmstudio import LLM
from pprint import pprint

In [6]:
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
from llmstudio_proxy.provider import ProxyConfig
proxy = ProxyConfig(host="0.0.0.0", port="8001")
print(proxy)

openai = LLM("openai", proxy_config=proxy)


host='0.0.0.0' port='8001' url=None username=None password=None
Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [7]:
openai._provider


### Chat (non-stream)

In [8]:
openai.chat("What's your name", model="gpt-4o")

ChatCompletion(id='dc5d3ebe-31a1-4f86-ab37-74ef17740be2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have a personal name, but you can call me Assistant. How can I help you today?", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729091060, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I don't have a personal name, but you can call me Assistant. How can I help you today?", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729091060.91643, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 26, 'cost_usd': 0.00035, 'latency_s': 0.7695848941802979, 'time_to_first_token_s': 0.5569219589233398, 'inter_token_latency_s': 0.009642221710898659, 'tokens_per_second': 29.88624149710970

#### Async version

In [9]:
await openai.achat("What's your name", model="gpt-4o")

ChatCompletion(id='2ae2738f-deb9-48dd-af77-590570029ba4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have a personal name, but you can call me Assistant! How can I assist you today?", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729091063, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I don't have a personal name, but you can call me Assistant! How can I assist you today?", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729091064.392381, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 26, 'cost_usd': 0.00035, 'latency_s': 1.7545521259307861, 'time_to_first_token_s': 0.8596808910369873, 'inter_token_latency_s': 0.040647235783663666, 'tokens_per_second': 13.108758446

### Chat (stream)

In [10]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, an infinite expanse that stretches beyond the confines of our planet, is a vast and

 mysterious frontier that has fascinated humanity for centuries. It is a realm where stars twinkle and galaxies swirl

, each a cosmic masterpiece, and where the laws of physics often defy our earthly understanding. Exploration

 of this enigmatic domain has led to remarkable scientific discoveries and technological advancements, from the moon landings to

 the deployment of telescopes that peer into the furthest reaches of the universe. Space is not only

 the home of planets and celestial bodies but also a crucible for the fundamental questions of existence, challenging

 us to ponder our place in the cosmos. The vastness of space, with its myriad wonders and

 potential for future exploration, continues to inspire awe and drive the quest for knowledge, pushing the boundaries of

 what we know and inviting us to explore the unknown.

## Metrics:
{'cost_usd': 0.0025900000000000003,
 'input_to

#### Async version

In [11]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, the vast and enigmatic expanse beyond our planet, captivates the imagination of humanity with

 its infinite possibilities and mysteries. Stretching billions of light-years, it is the realm where galaxies dance

, stars are born, and black holes lurk, challenging our understanding of the universe. The vacuum

 of space, nearly devoid of matter, creates a backdrop for celestial phenomena that evoke wonder and curiosity.

 From the swirling arms of spiral galaxies to the delicate hues of nebulae, space is a canvas

 of extraordinary beauty shaped over billions of years. As we venture further into this cosmic frontier with advanced technology

, we unearth clues about the origins of the universe, the potential for extraterrestrial life, and

 the fundamental laws that govern existence itself, reminding us of our small yet significant place in the cosmos.



## Metrics:
{'cost_usd': 9.779999999999999e-05,
 'input_tokens': 8,
 'inter_token_latency_s': 0.08299692422151565,
 'latency